## ライブラリのインポート

In [10]:
from typing import Final
import pandas as pd 
import csv
# import glob
import re
from IPython.display import display

from typing import List, Tuple, Dict
from nptyping import NDArray, Object

import warnings
warnings.simplefilter('ignore')

# ライブラリの設定変更
pd.options.display.max_rows = None


## フォルダ変数の設定

In [11]:
DATA_FILE_PATH: Final[str]="../0.input_data/"
DATA_FILE_OUTPUT_PATH: Final[str]="../2.output_data/"

# データの読み込み(小売物価統計調査（動向編）_全国)

In [12]:
with open(DATA_FILE_PATH + \
    "00200_0003449369_小売物価統計調査（動向編）_全国.csv",\
    "r", encoding="utf-8", errors="", newline="" ) as f:
    csv_result = csv.reader(f, delimiter=",")
    df_price_zen_1 = pd.DataFrame(csv_result)
#上部27行は削除する
df_price_zen_2 = df_price_zen_1.loc[27:]
# カラム名を整える
li_col_name:List[str] = [col for col in df_price_zen_2.loc[27]]
df_price_zen_2.columns = li_col_name
df_price_zen_3 = df_price_zen_2.iloc[1:]

# valueが"..."や"-"の場合削除する

df_price_zen_4 = df_price_zen_3[~((df_price_zen_3["value"]=="...")|(df_price_zen_3["value"]=="-"))]

# valueのデータ型をfloatにする
df_price_zen_4["value"] = df_price_zen_4["value"] .astype("float")

# df_price_zenというデータフレームに格納する
df_price_zen = df_price_zen_4.copy()
# データの確認

df_price_zen.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 39151 entries, 28 to 40023
Data columns (total 13 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   tab_code    39151 non-null  object 
 1   表章項目        39151 non-null  object 
 2   cat01_code  39151 non-null  object 
 3   銘柄          39151 non-null  object 
 4   cat02_code  39151 non-null  object 
 5   データの種別      39151 non-null  object 
 6   area_code   39151 non-null  object 
 7   地域          39151 non-null  object 
 8   time_code   39151 non-null  object 
 9   時間軸（月）      39151 non-null  object 
 10  unit        39151 non-null  object 
 11  value       39151 non-null  float64
 12  annotation  39151 non-null  object 
dtypes: float64(1), object(12)
memory usage: 4.2+ MB


In [13]:
# 各cat01_codeごとのデータ数を確認する
dic_cat01: Dict[str,int] = {}

for k,v in zip(df_price_zen["cat01_code"].value_counts().keys(),df_price_zen["cat01_code"].value_counts().values):
    dic_cat01[k] = v

# df_price_zenにカラム追加する
df_price_zen["n_data"] = df_price_zen["cat01_code"].map(dic_cat01)

print(df_price_zen["n_data"].value_counts())


97     8245
276    7728
179    4654
119    1785
156    1248
228    1140
157    1099
242     968
266     798
83      664
143     572
61      549
263     526
253     506
84      504
71      497
120     480
145     435
23      414
60      360
178     356
171     342
153     306
94      282
268     268
265     265
259     259
241     241
217     217
213     213
104     208
203     203
188     188
187     187
186     186
176     176
168     168
162     162
154     154
152     152
140     140
127     127
63      126
56      112
111     111
37      111
108     108
105     105
50      100
88       88
82       82
35       70
53       53
24       48
26       26
10       20
13       13
2         6
Name: n_data, dtype: int64


In [14]:

# 銘柄に記載されている内容を正規表現で抽出する【】で囲われている中身

str_test = "9373 遊園地入園料（小人）（大人）【2011年12月調査終了】【2004年12月以前は主要品目の】"

pattern = '.*?【(.*?)】.*?'

# findall 
results = re.findall(pattern, str_test)

print(results)

for result in results:
    str_test = str_test.replace(result,"").replace("【】","")

li_str_split = str_test.split(" ")

m_cat = li_str_split[1].split("（")[0]
s_cat = li_str_split[1].split("（")[1:]
s_cat2 = [i.replace("）","") for i in s_cat]


str_cat2 = ""
for id, s_cat in enumerate(s_cat2):
    if id==0 :
        str_cat2 = str_cat2 + s_cat
    else:
        str_cat2 = str_cat2 + "," + s_cat
    print(str_cat2)


print(results,li_str_split)

print(m_cat,str_cat2)



['2011年12月調査終了', '2004年12月以前は主要品目の']
小人
小人,大人
['2011年12月調査終了', '2004年12月以前は主要品目の'] ['9373', '遊園地入園料（小人）（大人）']
遊園地入園料 小人,大人


In [15]:
# テストしたことを関数化する

def regex_hoki(content:str) -> str:

    pattern = '.*?(【.*?】).*?'

    # findall
    results = re.findall(pattern,content)

    li_result:List[str] =[]
    str_cat2:str = ""
    m_cat:List[str] = []
    s_cat:List[str] = []

    for result in results:
        content = content.replace(result,"")
        result = result.replace("【","").replace("】","")
        li_result.append(result)

    li_content = content.split(" ")
    m_cat = li_content[1].split("（")[0]

    if len(li_content)>1:
        s_cat = li_content[1].split("（")[1:]
        s_cat2 = [i.replace("）","") for i in s_cat]

        str_cat2 = ""
        for id, s_cat in enumerate(s_cat2):
            if id==0 :
                str_cat2 = str_cat2 + s_cat
            else:
                str_cat2 = str_cat2 + "," + s_cat

    return li_result, m_cat, str_cat2

def price_range(value):
    if value <1 :
        return "単価1円未満"
    elif value >=1 and value <10:
        return "単価1円以上10円未満"
    elif value >=10 and value <100:
        return "単価10円以上100円未満"
    elif value >=100 and value <1000:
        return "単価100円以上1,000円未満"
    elif value >=1000 and value <10000:
        return "単価1,000円以上10,000円未満"
    elif value >=10000 and value <100000:
        return "単価10,000円以上100,000円未満"
    elif value >=100000 and value <1000000:
        return "単価100,000円以上1,000,000円未満"
    elif value >=1000000 and value <10000000:
        return "単価1,000,000円以上10,000,000円未満"
    else:
        return "単価10,000,000円以上"
    

df_price_zen["銘柄_cat01"] = df_price_zen["銘柄"].apply(lambda x: regex_hoki(x)[1])
df_price_zen["銘柄_cat02"] = df_price_zen["銘柄"].apply(lambda x: regex_hoki(x)[2])
df_price_zen["銘柄_add"] = df_price_zen["銘柄"].apply(lambda x: regex_hoki(x)[0])
df_price_zen["銘柄_add_len"] = df_price_zen["銘柄"].apply(lambda x: len(regex_hoki(x)[0]))
df_price_zen["価格帯_単価"] = df_price_zen["value"].apply(lambda x: price_range(x))
    

#display(df_price_zen.loc[df_price_zen["銘柄_prep_hoki"]=="[【2014年12月以前は主要品目の都市別小売価格に表章】]",["銘柄","銘柄_prep","銘柄_prep_hoki"]].head())
#print(df_price_zen["銘柄_prep_hoki_len"].value_counts())
display(df_price_zen.loc[df_price_zen["銘柄_add_len"]==2,["銘柄","銘柄_cat01","銘柄_cat02","銘柄_add"]].head())
display(df_price_zen[["銘柄","銘柄_cat01","銘柄_cat02","銘柄_add","value","価格帯_単価"]].head())

,銘柄,銘柄_cat01,銘柄_cat02,銘柄_add
32789,9373 遊園地入園料（小人）【2011年12月調査終了】【2004年12月以前は主要品目の...,遊園地入園料,小人,"[2011年12月調査終了, 2004年12月以前は主要品目の都市別小売価格に表章]"
32790,9373 遊園地入園料（小人）【2011年12月調査終了】【2004年12月以前は主要品目の...,遊園地入園料,小人,"[2011年12月調査終了, 2004年12月以前は主要品目の都市別小売価格に表章]"
32791,9373 遊園地入園料（小人）【2011年12月調査終了】【2004年12月以前は主要品目の...,遊園地入園料,小人,"[2011年12月調査終了, 2004年12月以前は主要品目の都市別小売価格に表章]"
32792,9373 遊園地入園料（小人）【2011年12月調査終了】【2004年12月以前は主要品目の...,遊園地入園料,小人,"[2011年12月調査終了, 2004年12月以前は主要品目の都市別小売価格に表章]"
32793,9373 遊園地入園料（小人）【2011年12月調査終了】【2004年12月以前は主要品目の...,遊園地入園料,小人,"[2011年12月調査終了, 2004年12月以前は主要品目の都市別小売価格に表章]"


,銘柄,銘柄_cat01,銘柄_cat02,銘柄_add,value,価格帯_単価
28,1924 コーヒー飲料（セルフ式）,コーヒー飲料,セルフ式,[],110.0,"単価100円以上1,000円未満"
29,1924 コーヒー飲料（セルフ式）,コーヒー飲料,セルフ式,[],110.0,"単価100円以上1,000円未満"
30,1924 コーヒー飲料（セルフ式）,コーヒー飲料,セルフ式,[],110.0,"単価100円以上1,000円未満"
31,1924 コーヒー飲料（セルフ式）,コーヒー飲料,セルフ式,[],110.0,"単価100円以上1,000円未満"
32,1924 コーヒー飲料（セルフ式）,コーヒー飲料,セルフ式,[],105.0,"単価100円以上1,000円未満"


In [16]:
df_price_zen.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 39151 entries, 28 to 40023
Data columns (total 19 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   tab_code    39151 non-null  object 
 1   表章項目        39151 non-null  object 
 2   cat01_code  39151 non-null  object 
 3   銘柄          39151 non-null  object 
 4   cat02_code  39151 non-null  object 
 5   データの種別      39151 non-null  object 
 6   area_code   39151 non-null  object 
 7   地域          39151 non-null  object 
 8   time_code   39151 non-null  object 
 9   時間軸（月）      39151 non-null  object 
 10  unit        39151 non-null  object 
 11  value       39151 non-null  float64
 12  annotation  39151 non-null  object 
 13  n_data      39151 non-null  int64  
 14  銘柄_cat01    39151 non-null  object 
 15  銘柄_cat02    39151 non-null  object 
 16  銘柄_add      39151 non-null  object 
 17  銘柄_add_len  39151 non-null  int64  
 18  価格帯_単価      39151 non-null  object 
dtypes: float64(1), int64(2),

In [17]:
ddf_price_zen = df_price_zen.sort_values(by=["cat01_code","time_code"],ascending=[True,True])


In [18]:
# ファイルを出力
# Tableau用
df_price_zen.to_csv(DATA_FILE_OUTPUT_PATH + "data_price_zen.csv",index=False,encoding="utf_8")
# 目検用
df_price_zen.to_csv(DATA_FILE_OUTPUT_PATH + "data_price_zen_shift_jis.csv",index=False,encoding="CP932")

## データの読み込み（2020年基準消費者物価指数）

In [19]:
with open(DATA_FILE_PATH + \
    "00200_0003427113_2020年基準消費者物価指数_消費者物価指数（2020年基準）.csv",\
    "r", encoding="shift_jis", errors="", newline="" ) as f:
    csv_result = csv.reader(f, delimiter=",")
    df_price_idx_1 = pd.DataFrame(csv_result)
#上部27行は削除する
df_price_idx_2 = df_price_idx_1.loc[27:]
# カラム名を整える
li_col_name:List[str] = [col for col in df_price_idx_2.loc[27]]
df_price_idx_2.columns = li_col_name
df_price_idx_3 = df_price_idx_2.iloc[1:]

# valueが"..."や"-"の場合削除する

df_price_idx_4 = df_price_idx_3[~((df_price_idx_3["value"]=="...")|(df_price_idx_3["value"]=="-"))]

# valueのデータ型をfloatにする
df_price_idx_4["value"] = df_price_idx_4["value"] .astype("float")

# df_price_idxというデータフレームに格納する
df_price_idx = df_price_idx_4.copy()
# データの確認

df_price_idx.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100000 entries, 28 to 100027
Data columns (total 11 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   tab_code     100000 non-null  object 
 1   表章項目         100000 non-null  object 
 2   cat01_code   100000 non-null  object 
 3   2020年基準品目    100000 non-null  object 
 4   area_code    100000 non-null  object 
 5   地域（2020年基準）  100000 non-null  object 
 6   time_code    100000 non-null  object 
 7   時間軸（年・月）     100000 non-null  object 
 8   unit         100000 non-null  object 
 9   value        100000 non-null  float64
 10  annotation   100000 non-null  object 
dtypes: float64(1), object(10)
memory usage: 9.2+ MB


In [20]:
# まず2022年の指数を取り出して新しいカラムにくっつける

def true_false_match_year(x):
    pattern1 = "[0-9]+年$"
    pattern2 = "[0-9]+年度$"
    if re.match(pattern1,x) or re.match(pattern2,x):
        return True
    else:
        return False

def true_false_match_area(x):
    pattern = ".+地方$"
    if re.match(pattern,x):
        return "area_cat"
    else:
        return ""
    
#print(true_false_match("2021年度"))

df_price_idx["年次_flag"] = df_price_idx["時間軸（年・月）"].apply(lambda x : true_false_match_year(x))
df_price_idx_year = df_price_idx.loc[df_price_idx["年次_flag"]==True]
df_price_idx_month = df_price_idx.loc[df_price_idx["年次_flag"]==False]
df_price_idx_month["時間軸（年）"] = df_price_idx_month["時間軸（年・月）"].apply(lambda x: str(x)[0:5])
df_price_idx_month = df_price_idx_month.rename(columns={"時間軸（年・月）":"時間軸（月）"})
df_price_idx_year = df_price_idx_year.rename(columns={"時間軸（年・月）":"時間軸（年）"})

df_price_idx_time = pd.merge(df_price_idx_month,df_price_idx_year[["cat01_code","area_code","時間軸（年）","value"]],how="left", on=["cat01_code","area_code","時間軸（年）"])
#display(df_price_idx_time.head())

list_city_cat:List[str] = ["大都市","中都市","小都市Ｂ・町村","小都市Ａ"]
list_all_cat:List[str] = ["全国"]

df_price_idx_time["area_cat01"] = ""
df_price_idx_time["area_cat01"] = df_price_idx_time["地域（2020年基準）"].apply(lambda x: true_false_match_area(x)) 
df_price_idx_time.loc[df_price_idx_time["地域（2020年基準）"].isin(list_city_cat),"area_cat01"] = "city_cat_01"
df_price_idx_time.loc[df_price_idx_time["地域（2020年基準）"].isin(list_all_cat),"area_cat01"] = "all_cat"
df_price_idx_time.loc[df_price_idx_time["area_cat01"]=="","area_cat01"] = "city_cat_02"
df_price_idx_time = df_price_idx_time.rename(columns={"value_x":"物価指数値_月次","value_y":"物価指数値_年次"})

display(df_price_idx_time.head())

# df_price_idx_year = df_price_idx[""]

,tab_code,表章項目,cat01_code,2020年基準品目,area_code,地域（2020年基準）,time_code,時間軸（月）,unit,物価指数値_月次,annotation,年次_flag,時間軸（年）,物価指数値_年次,area_cat01
0,1,指数,0001,0001 総合,13A01,13100 東京都区部,2023000101,2023年1月,,104.7,,False,2023年,NaN,city_cat_02
1,1,指数,0001,0001 総合,13A01,13100 東京都区部,2022001212,2022年12月,,104.0,,False,2022年,102.2,city_cat_02
2,1,指数,0001,0001 総合,13A01,13100 東京都区部,2022001111,2022年11月,,103.8,,False,2022年,102.2,city_cat_02
3,1,指数,0001,0001 総合,13A01,13100 東京都区部,2022001010,2022年10月,,103.5,,False,2022年,102.2,city_cat_02
4,1,指数,0001,0001 総合,13A01,13100 東京都区部,2022000909,2022年9月,,102.9,,False,2022年,102.2,city_cat_02


In [21]:
# ファイルを出力
# Tableau用
df_price_idx_time.to_csv(DATA_FILE_OUTPUT_PATH + "data_price_idx.csv",index=False,encoding="utf_8")
# 目検用
df_price_idx_time.to_csv(DATA_FILE_OUTPUT_PATH + "data_price_idx_shift_jis.csv",index=False,encoding="CP932")

# データの読み込み(小売物価統計調査（構造編）_地域別価格差 10大費目別消費者物価地域差指数 （全国平均＝100）)

In [26]:
with open(DATA_FILE_PATH + \
    "00200_0003441258_小売物価統計調査（構造編）_地域別価格差 10大費目別消費者物価地域差指数 （全国平均＝100）－全国，地方，都道府県，都道府県庁所在市及び政令指定都市（2013年～）.csv",\
    "r", encoding="shift_jis", errors="", newline="" ) as f:
    csv_result = csv.reader(f, delimiter=",")
    df_kouzo_idx_1 = pd.DataFrame(csv_result)
#上部27行は削除する
df_kouzo_idx_2 = df_kouzo_idx_1.loc[27:]
# カラム名を整える
li_col_name:List[str] = [col for col in df_kouzo_idx_2.loc[27]]
df_kouzo_idx_2.columns = li_col_name
df_kouzo_idx_3 = df_kouzo_idx_2.iloc[1:]

# valueが"..."や"-"の場合削除する
print(len(df_kouzo_idx_3))
df_kouzo_idx_4 = df_kouzo_idx_3[~((df_kouzo_idx_3["value"]=="...")|(df_kouzo_idx_3["value"]=="-"))]
print(len(df_kouzo_idx_4))

# valueのデータ型をfloatにする
df_kouzo_idx_4["value"] = df_kouzo_idx_4["value"] .astype("float")

# df_price_idxというデータフレームに格納する
df_kouzo_idx = df_kouzo_idx_4.copy()
# データの確認

df_kouzo_idx.info()

11244
7509
<class 'pandas.core.frame.DataFrame'>
Int64Index: 7509 entries, 28 to 11271
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   tab_code    7509 non-null   object 
 1   表章項目        7509 non-null   object 
 2   cat01_code  7509 non-null   object 
 3   10大費目       7509 non-null   object 
 4   area_code   7509 non-null   object 
 5   地域          7509 non-null   object 
 6   time_code   7509 non-null   object 
 7   時間軸(年)      7509 non-null   object 
 8   unit        7509 non-null   object 
 9   value       7509 non-null   float64
 10  annotation  7509 non-null   object 
dtypes: float64(1), object(10)
memory usage: 704.0+ KB


In [27]:
list_pre:List[str] = ["北海道","青森県","岩手県","宮城県","秋田県","山形県","福島県","茨城県","栃木県", \
                      "群馬県","埼玉県","千葉県","東京都","神奈川県","新潟県","富山県","石川県","福井県","山梨県","長野県", \
                        "岐阜県","静岡県","愛知県","三重県","滋賀県","京都府","大阪府","兵庫県","奈良県","和歌山県","鳥取県", \
                            "島根県","岡山県","広島県","山口県","徳島県","香川県","愛媛県","高知県","福岡県","佐賀県","長崎県", \
                                "熊本県","大分県","宮崎県","鹿児島県","沖縄県"]

print(f"処理前：{len(df_kouzo_idx)}")
df_kouzo_idx_pre_1 = df_kouzo_idx.loc[df_kouzo_idx["地域"].isin(list_pre)]

print(f"処理後：{len(df_kouzo_idx_pre_1)}") # 47×9×12=5076

df_kouzo_idx_pre_1 = df_kouzo_idx_pre_1.rename(columns={"value":"都道府県_value","地域":"都道府県"})
df_kouzo_idx_pre_1["全国_value"] = 100
df_kouzo_idx_pre_1["差分(都道府県-全国)"] = df_kouzo_idx_pre_1["都道府県_value"]-df_kouzo_idx_pre_1["全国_value"] 
df_kouzo_idx_pre_2 = df_kouzo_idx_pre_1.reindex(columns=["area_code","都道府県","time_code","時間軸(年)","都道府県_value","全国_value","cat01_code","10大費目","差分(都道府県-全国)"])

処理前：7509
処理後：5076


In [28]:
df_kouzo_idx_pre_2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5076 entries, 127 to 10856
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   area_code    5076 non-null   object 
 1   都道府県         5076 non-null   object 
 2   time_code    5076 non-null   object 
 3   時間軸(年)       5076 non-null   object 
 4   都道府県_value   5076 non-null   float64
 5   全国_value     5076 non-null   int64  
 6   cat01_code   5076 non-null   object 
 7   10大費目        5076 non-null   object 
 8   差分(都道府県-全国)  5076 non-null   float64
dtypes: float64(2), int64(1), object(6)
memory usage: 396.6+ KB


In [29]:
df_kouzo_idx_pre= df_kouzo_idx_pre_2.sort_values(by=["area_code","time_code","cat01_code"],ascending=[True,True,True])
display(df_kouzo_idx_pre.head())

,area_code,都道府県,time_code,時間軸(年),都道府県_value,全国_value,cat01_code,10大費目,差分(都道府県-全国)
135,01000,北海道,2013000000,2013年,99.2,100,00010,総合,-0.8
1072,01000,北海道,2013000000,2013年,99.0,100,00020,食料,-1.0
2009,01000,北海道,2013000000,2013年,88.8,100,00030,住居,-11.2
2946,01000,北海道,2013000000,2013年,108.8,100,00040,光熱・水道,8.8
3883,01000,北海道,2013000000,2013年,97.8,100,00050,家具・家事用品,-2.2


In [30]:
# ファイルを出力
# Tableau用
df_kouzo_idx_pre.to_csv(DATA_FILE_OUTPUT_PATH + "data_10th_item_idx_pre.csv",index=False,encoding="utf_8")
# 目検用
df_kouzo_idx_pre.to_csv(DATA_FILE_OUTPUT_PATH + "data_10th_item_idx_pre_shift_jis.csv",index=False,encoding="CP932")

# データ読み込み（小売物価統計調査（構造編）_地域別価格差 調査品目の地域差指数算出用年平均価格－全国，地方，都道府県（2013年～2019年）

In [31]:
with open(DATA_FILE_PATH + \
    "00200_0003441558_小売物価統計調査（構造編）_地域別価格差 調査品目の地域差指数算出用年平均価格－全国，地方，都道府県（2013年～2019年）.csv",\
    "r", encoding="shift_jis", errors="", newline="" ) as f:
    csv_result = csv.reader(f, delimiter=",")
    df_hinmoku_idx_1 = pd.DataFrame(csv_result)
#上部27行は削除する
df_hinmoku_idx_2 = df_hinmoku_idx_1.loc[27:]
# カラム名を整える
li_col_name:List[str] = [col for col in df_hinmoku_idx_2.loc[27]]
df_hinmoku_idx_2.columns = li_col_name
df_hinmoku_idx_3 = df_hinmoku_idx_2.iloc[1:]

# valueが"..."や"-"の場合削除する
print(len(df_hinmoku_idx_3))
df_hinmoku_idx_4 = df_hinmoku_idx_3[~((df_hinmoku_idx_3["value"]=="...")|(df_hinmoku_idx_3["value"]=="-"))]
print(len(df_hinmoku_idx_4))

# valueのデータ型をfloatにする
df_hinmoku_idx_4["value"] = df_hinmoku_idx_4["value"] .astype("float")

# df_price_idxというデータフレームに格納する
df_hinmoku_idx = df_hinmoku_idx_4.copy()
# データの確認

df_hinmoku_idx.info()

47096
47052
<class 'pandas.core.frame.DataFrame'>
Int64Index: 47052 entries, 28 to 47123
Data columns (total 13 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   tab_code    47052 non-null  object 
 1   表章項目        47052 non-null  object 
 2   cat01_code  47052 non-null  object 
 3   銘柄          47052 non-null  object 
 4   cat02_code  47052 non-null  object 
 5   平均価格        47052 non-null  object 
 6   area_code   47052 non-null  object 
 7   地域          47052 non-null  object 
 8   time_code   47052 non-null  object 
 9   時間軸(年)      47052 non-null  object 
 10  unit        47052 non-null  object 
 11  value       47052 non-null  float64
 12  annotation  47052 non-null  object 
dtypes: float64(1), object(12)
memory usage: 5.0+ MB


In [32]:
list_pre:List[str] = ["北海道","青森県","岩手県","宮城県","秋田県","山形県","福島県","茨城県","栃木県", \
                      "群馬県","埼玉県","千葉県","東京都","神奈川県","新潟県","富山県","石川県","福井県","山梨県","長野県", \
                        "岐阜県","静岡県","愛知県","三重県","滋賀県","京都府","大阪府","兵庫県","奈良県","和歌山県","鳥取県", \
                            "島根県","岡山県","広島県","山口県","徳島県","香川県","愛媛県","高知県","福岡県","佐賀県","長崎県", \
                                "熊本県","大分県","宮崎県","鹿児島県","沖縄県"]

print(f"処理前：{len(df_hinmoku_idx)}")
df_hinmoku_idx_pre_1 = df_hinmoku_idx.loc[df_hinmoku_idx["地域"].isin(list_pre)]
df_hinmoku_idx_pre_all = df_hinmoku_idx.loc[df_hinmoku_idx["地域"]=="全国"]

print(f"処理後：{len(df_hinmoku_idx_pre_1)}") # 47×9×12=5076
print(f"処理後：{len(df_hinmoku_idx_pre_all)}")

df_hinmoku_idx_pre_1 = df_hinmoku_idx_pre_1.rename(columns={"value":"都道府県_value","地域":"都道府県"})
df_hinmoku_idx_pre_all = df_hinmoku_idx_pre_all.rename(columns={"value":"全国_value","地域":"全国"})

#display(df_hinmoku_idx_pre_1.head())
#display(df_hinmoku_idx_pre_all.head())

df_hinmoku_pre_1 = pd.merge(df_hinmoku_idx_pre_1,df_hinmoku_idx_pre_all[["cat01_code","cat02_code","time_code","全国_value"]],how="left",on=["cat01_code","cat02_code","time_code"])

#display(df_hinmoku_pre_1.head())
df_hinmoku_pre_1["差分(都道府県-全国)"] = df_hinmoku_pre_1["都道府県_value"]-df_hinmoku_pre_1["全国_value"] 
df_hinmoku_pre_2 = df_hinmoku_pre_1.reindex(columns=["area_code","都道府県","cat01_code","銘柄","cat02_code","平均価格","time_code","時間軸(年)","全国_value","都道府県_value","差分(都道府県-全国)"])

display(df_hinmoku_pre_2.head())

処理前：47052
処理後：38142
処理後：812


,area_code,都道府県,cat01_code,銘柄,cat02_code,平均価格,time_code,時間軸(年),全国_value,都道府県_value,差分(都道府県-全国)
0,01000,北海道,01001,1001 うるち米(単一品種，「コシヒカリ」),00010,算術平均,2019000000,2019年,2319.0,2356.0,37.0
1,01000,北海道,01001,1001 うるち米(単一品種，「コシヒカリ」),00010,算術平均,2018000000,2018年,2303.0,2361.0,58.0
2,01000,北海道,01001,1001 うるち米(単一品種，「コシヒカリ」),00010,算術平均,2017000000,2017年,2223.0,2289.0,66.0
3,01000,北海道,01001,1001 うるち米(単一品種，「コシヒカリ」),00010,算術平均,2016000000,2016年,2156.0,2193.0,37.0
4,01000,北海道,01001,1001 うるち米(単一品種，「コシヒカリ」),00010,算術平均,2015000000,2015年,2082.0,2162.0,80.0


In [33]:
df_hinmoku_pre_2.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 38142 entries, 0 to 38141
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   area_code    38142 non-null  object 
 1   都道府県         38142 non-null  object 
 2   cat01_code   38142 non-null  object 
 3   銘柄           38142 non-null  object 
 4   cat02_code   38142 non-null  object 
 5   平均価格         38142 non-null  object 
 6   time_code    38142 non-null  object 
 7   時間軸(年)       38142 non-null  object 
 8   全国_value     38142 non-null  float64
 9   都道府県_value   38142 non-null  float64
 10  差分(都道府県-全国)  38142 non-null  float64
dtypes: float64(3), object(8)
memory usage: 3.5+ MB


In [34]:
df_hinmoku_pre_2 = df_hinmoku_pre_2.sort_values(by=["area_code","cat01_code","cat02_code","time_code"],ascending=[True,True,True,True])

In [35]:
# ファイルを出力
# Tableau用
df_hinmoku_pre_2.to_csv(DATA_FILE_OUTPUT_PATH + "data_many_items_price_pre.csv",index=False,encoding="utf_8")
# 目検用
df_hinmoku_pre_2.to_csv(DATA_FILE_OUTPUT_PATH + "data_many_items_price_pre_shift_jis.csv",index=False,encoding="CP932")

In [36]:
df_hinmoku_pre_2.head() 

,area_code,都道府県,cat01_code,銘柄,cat02_code,平均価格,time_code,時間軸(年),全国_value,都道府県_value,差分(都道府県-全国)
6,01000,北海道,01001,1001 うるち米(単一品種，「コシヒカリ」),00010,算術平均,2013000000,2013年,2370.0,2345.0,-25.0
5,01000,北海道,01001,1001 うるち米(単一品種，「コシヒカリ」),00010,算術平均,2014000000,2014年,2223.0,2212.0,-11.0
4,01000,北海道,01001,1001 うるち米(単一品種，「コシヒカリ」),00010,算術平均,2015000000,2015年,2082.0,2162.0,80.0
3,01000,北海道,01001,1001 うるち米(単一品種，「コシヒカリ」),00010,算術平均,2016000000,2016年,2156.0,2193.0,37.0
2,01000,北海道,01001,1001 うるち米(単一品種，「コシヒカリ」),00010,算術平均,2017000000,2017年,2223.0,2289.0,66.0
